# Loading CZI Metadata

A notebook to test on metadata reading using different readers

In [24]:
import numpy as np
from aicspylibczi import CziFile
from pathlib import Path
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
from xml.etree.ElementTree import ElementTree, Element
import json

In [2]:
IMAGE_DATA = "/mnt/Imaging/Idrive/Group Fritzsche/Jacky/myocardial cells/For sharing analysis/no097_FR_D30_Q2_MT.czi"

## aicspylibczi
A demo code to load CZI metadata using [aicspylibczi](https://github.com/AllenCellModeling/aicspylibczi)

In [3]:
# Read the original image
czi = CziFile(Path(IMAGE_DATA))

# Get the shape of the data, the coordinate pairs are (start index, size)
print("Dimensions:", czi.get_dims_shape())
print("Dimension Order: ", czi.dims)
print("Is Mosaic:", czi.is_mosaic())

Dimensions: [{'A': (0, 3), 'X': (0, 1600), 'Y': (0, 1200), 'C': (0, 1), 'M': (0, 300), 'S': (0, 1)}, {'A': (0, 3), 'X': (0, 1600), 'Y': (0, 1200), 'C': (0, 1), 'M': (0, 270), 'S': (1, 2)}]
Dimension Order:  SCMYXA
Is Mosaic: True


### Get Metadata Object

The image metadata is returned as a XML element tree object. It would be easier to manipulate in python with dictionary format, here we show the way for metadata extraction and saving.

In [20]:
META = czi.meta

In [8]:
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v
                     for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v)
                        for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
                d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [21]:
d = etree_to_dict(META)

pprint(d)

{'ImageDocument': {'Metadata': {'AttachmentInfos': {'AttachmentInfo': {'@Id': 'Label:1',
                                                                       'Label': {'Barcodes': {'Barcode': {'@Id': 'Barcode:1',
                                                                                                          'Content': None,
                                                                                                          'Type': None}},
                                                                                 'OCRs': {'OCR': {'@Id': 'OCR:1',
                                                                                                  'Content': None}}},
                                                                       'Profile': {'IsProfileModified': 'False',
                                                                                   'ProfileName': 'REF_BF '
                                                                                       

In [26]:
# save the metadata to local
ElementTree(META).write("./no097_FR_D30_Q2_MT.xml")
with open("./no097_FR_D30_Q2_MT.json", "w") as fp:
    json.dump(d,fp, indent = 4) 

### Extract Important Image Informations

In [44]:
# image information metadata
pprint(d["ImageDocument"]["Metadata"]["Information"]["Image"])


{'AcquisitionDateAndTime': '2022-11-08T13:04:46.7054438Z',
 'AcquisitionDuration': '264161.16120000003',
 'ComponentBitCount': '8',
 'Dimensions': {'Channels': {'Channel': {'@Id': 'Channel:0',
                                         '@Name': 'TL Brightfield',
                                         'AcquisitionMode': 'WideField',
                                         'Color': '#FFFFFFFF',
                                         'ComponentBitCount': '8',
                                         'ContrastMethod': 'Brightfield',
                                         'CustomAttributes': {'MM.TotalMagnification': '19.96678'},
                                         'DetectorSettings': {'Binning': '1,1',
                                                              'Detector': {'@Id': 'Detector:Hitachi '
                                                                                  'HV-F203SCL'}},
                                         'ExposureTime': '200000',
               

In [43]:
# pixel scaling
pprint(d["ImageDocument"]["Metadata"]["Scaling"]["Items"]["Distance"])

[{'@Id': 'X', 'DefaultUnitFormat': 'µm', 'Value': '2.2036602797246227E-07'},
 {'@Id': 'Y', 'DefaultUnitFormat': 'µm', 'Value': '2.2036602797246227E-07'}]
